In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pyfusion.algs import eed
from pyfusion.data import load
from pyfusion.structs import math
from pyfusion.structs.validation_utils import FloatArr
from pyfusion.utils import projection

In [ ]:
SLICE = 70

In [ ]:
MAX_ITER = 5000
STEP_SIZE = 0.1

In [ ]:
FSL = load.load_fsl("../data/example_mri", SLICE)

In [ ]:
# type: ignore
# Plotting the FA map
plt.figure(figsize=(10, 8))
plt.imshow(FSL.FA, cmap="gray")
plt.title("Fractional Anisotropy (FA) Map")
plt.axis("off")
plt.colorbar()
plt.show()

In [ ]:
# add random noise to the FA map
FA_noisy = FSL.FA + 0.3 * FSL.FA.std() * np.random.randn(*FSL.FA.shape)

In [ ]:
# type: ignore
# Plotting the noisy FA map
plt.figure(figsize=(10, 8))
plt.imshow(FA_noisy, cmap="gray")
plt.title("Noisy")
plt.axis("off")
plt.colorbar()
plt.show()

In [ ]:
eigen_1 = math.Eigen3D(eigvec=FSL.V1, eigval=FSL.L1)
eigen_2 = math.Eigen3D(eigvec=FSL.V2, eigval=FSL.L2)
eigen_3 = math.Eigen3D(eigvec=FSL.V3, eigval=FSL.L3)

eigen_decomp = math.EigenDecomp3D(
    eigen_1=eigen_1,
    eigen_2=eigen_2,
    eigen_3=eigen_3,
)

In [ ]:
def heat_conduction(x: FloatArr) -> FloatArr:
    """
    Heat conduction function for the diffusion tensor.
    """
    return 1 / (1 + x**2)

In [ ]:
tensor_field = projection.calculate_eigen3d(eigen_decomp)

decomposed_tensor = projection.calculate_decomposition(tensor_field)

heated_tensor = eed.apply_heat_conduction(decomposed_tensor, heat_conduction)

In [ ]:
img = math.Image(data=FA_noisy.copy())

for _ in range(MAX_ITER):
    img = eed.step(img, tensor_field, step_size=STEP_SIZE)

In [ ]:
# type: ignore
plt.figure(figsize=(10, 8))
plt.imshow(img.data, cmap="gray")
plt.title("Diffusion Tensor Imaging (DTI) After EED")
plt.axis("off")
plt.colorbar()
plt.show()

In [ ]:
# type: ignore
difference = FSL.FA - img.data
plt.figure(figsize=(10, 8))
plt.imshow(difference, cmap="gray")
plt.title("Difference between FA and DTI Step")
plt.axis("off")
plt.colorbar()
plt.show()